# Trail Experiment to do  Machine Learning on Blockchain(BlockFL)


#### Imports

In [0]:
import pandas as pd
import numpy as np
import json
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from flask import Flask, Response, request, jsonify
from web3 import Web3

# How to share configuration

Decide on the configuration that should be used by owner ,user, as well as splitting the data to be used by each.

#### Data

In [2]:
input = pd.read_csv('Iris.csv')
input.head()

In [0]:
# Splitting between training and testing
X_train, X_test, y_train, y_test = train_test_split(input.loc[:,input.columns != 'Species'], input['Species'], test_size=0.2)

#### Setting up configurations to access smart contract

In [0]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
        
contract_abi = '[{"constant": false,"inputs": [{"name": "_encoded_parameters","type": "string"}],"name": "setModelParameters","outputs": [],"payable": false,"stateMutability": "nonpayable","type": "function"},{"inputs": [],"payable": false,"stateMutability": "nonpayable","type": "constructor"},{"constant": true,"inputs": [],"name": "getModelParameters","outputs": [{"name": "_encoded_parameters","type": "string"}],"payable": false,"stateMutability": "view","type": "function"}]'
contract_address = Web3.toChecksumAddress('0x748b9e15ac55f06bdd4b605fa6d1be5d1e7c2ed7')

ml_contract = w3.eth.contract(address=contract_address, abi=contract_abi)

# Section 1: Miner

This is the part that will be used by the miner. Here, the model is trained on private data, and the model parameters are uploaded to the smart contract. We should ensure that the address of the user uploading this data is the same as the owner of the smart contract.

### Training

In [6]:
cls = LogisticRegression()
f = cls.fit(X_train,y_train)
cls.coef_

### Encode model parameters to json

In [7]:
encoded = json.dumps((cls.coef_.tolist(), cls.intercept_.tolist(), cls.classes_.tolist()))
encoded

### Upload to Smart Contract

In [0]:
w3.eth.defaultAccount = w3.eth.accounts[0]
    
tx_hash = ml_contract.functions.setModelParameters(
    encoded
)
tx_hash = tx_hash.transact()

# Wait for transaction to be mined...
w3.eth.waitForTransactionReceipt(tx_hash)

# Module 2: User

Now we will check how the user can retrieve the model parameters from the smart contract, and create a logistic regression model based on them. 

### Retrieving data from smart contract and decoding

In [0]:
w3.eth.defaultAccount = w3.eth.accounts[1]

encoded_parameters = ml_contract.functions.getModelParameters().call()
decoded_parameters = json.loads(encoded_parameters)
decoded_parameters

### Creating model based on parameters retrieved

In [0]:
cls = LogisticRegression()
cls.coef_ = np.array(decoded_parameters[0])
cls.intercept_ = np.array(decoded_parameters[1])
cls.classes_ = np.array(decoded_parameters[2])

### Performing Prediction

In [0]:
predictions = cls.predict(X_test);
result = pd.DataFrame(np.vstack((predictions, y_test)).T,columns=['Predicted Outcomes','Actual Outcomes'])
result.head()

### Showing resulting metrics

In [0]:
accuracy = np.sum(predictions == y_test) / y_test.shape[0] * 100
conf_matrix = confusion_matrix(predictions, y_test)
precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1]) * 100

print(conf_matrix)
print("Accuracy: {0:.2f}%".format(accuracy))
print("Precision: {0:.2f}%".format(precision))